# Using Language Models
First up, let's learn how to use a language model by itself. LangChain supports many different language models that you can use interchangably - select the one you want to use below!

In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"

os.environ["OPENAI_API_KEY"] = "sk-proj-5fx7Rml7AzN1lUyC0BmuT3BlbkFJZZjPShuYnYXZ7QZXBvyG"

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

Let's first use the model directly. ChatModels are instances of LangChain "Runnables", which means they expose a standard interface for interacting with them. To just simply call the model, we can pass in a list of messages to the .invoke method.

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='ciao!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fa70fa1d-7466-4177-af80-851c32472f9f-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23})

API Reference:HumanMessage | SystemMessage

```python
AIMessage(content='ciao!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fc5d7c88-9615-48ab-a3c7-425232b562c5-0')
```

If we've enable LangSmith, we can see that this run is logged to LangSmith, and can see the LangSmith trace

# Output Parsers

Notice that the response from the model is an AIMessage. This contains a string response along with other metadata about the response. Oftentimes we may just want to work with the string response. We can parse out just this response by using a simple output parser.

We first import the simple output parser.from langchain_core.output_parsers import StrOutputParser

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

**API Reference:** [StrOutputParser](https://api.python.langchain.com/en/latest/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html)

One way to use it is to use it by itself. For example, we could save the result of the language model call and then pass it to the parser.

In [9]:
result = model.invoke(messages)

In [10]:
parser.invoke(result)

'Ciao!'

More commonly, we can "chain" the model with this output parser. This means this output parser will get called everytime in this chain. This chain takes on the input type of the language model (string or list of message) and returns the output type of the output parser (string).

We can easily create the chain using the | operator. The | operator is used in LangChain to combine two elements together.

In [12]:
chain = model | parser

In [13]:
chain.invoke(messages)

'Ciao!'

If we now look at LangSmith, we can see that the chain has two steps: first the language model is called, then the result of that is passed to the output parser. We can see the [LangSmith trace](https://smith.langchain.com/public/f1bdf656-2739-42f7-ac7f-0f1dd712322f/r)

# Prompt Templates

Right now we are passing a list of messages directly into the language model. Where does this list of messages come from? Usually, it is constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model. Common transformations include adding a system message or formatting a template with the user input.

PromptTemplates are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model.

Let's create a PromptTemplate here. It will take in two user variables:

    language: The language to translate text into
    text: The text to translate

In [14]:
from langchain_core.prompts import ChatPromptTemplate

API Reference:[ChatPromptTemplate](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)

First, let's create a string that we will format to be the system message:

In [16]:
system_template = "Translate the following into {language}:"

Next, we can create the PromptTemplate. This will be a combination of the system_template as well as a simpler template for where the put the text

In [17]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

The input to this prompt template is a dictionary. We can play around with this prompt template by itself to see what it does by itself

In [18]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')])

We can see that it returns a ChatPromptValue that consists of two messages. If we want to access the messages directly we do:

In [19]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

# Chaining together components with LCEL

We can now combine this with the model and the output parser from above using the pipe (|) operator:

In [20]:
chain = prompt_template | model | parser

In [21]:
chain.invoke({"language": "italian", "text": "hi"})

'ciao'